In [1]:
import os

import numpy as np

_word_to_idx = {}
_idx_to_word = []


def _add_word(word):
    idx = len(_idx_to_word)
    _word_to_idx[word] = idx
    _idx_to_word.append(word)
    return idx

PAD_WORD = "<PAD>"
UNKNOWN_WORD = "<UNK>"
START_WORD = "<START>"
END_WORD = "<END>"
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
PAD_TOKEN = _add_word(PAD_WORD)
UNKNOWN_TOKEN = _add_word(UNKNOWN_WORD)
START_TOKEN = _add_word(START_WORD)
END_TOKEN = _add_word(END_WORD)


def look_up_word(word):
    return _word_to_idx.get(word, UNKNOWN_TOKEN)


def look_up_token(token):
    return _idx_to_word[token]


with open('/home/aditi/Desktop/6th sem/Computer Graphics/glove.6B.100d.txt', encoding='utf-8') as f:
    line = f.readline()
    chunks = line.split(" ")
    dimensions = len(chunks) - 1
    f.seek(0)

    vocab_size = sum(1 for line in f)
    vocab_size += 3
    f.seek(0)

    glove = np.ndarray((vocab_size, dimensions), dtype=np.float32)
    glove[PAD_TOKEN] = np.zeros(dimensions)
    glove[UNKNOWN_TOKEN] = np.zeros(dimensions)
    glove[START_TOKEN] = -np.ones(dimensions)
    glove[END_TOKEN] = np.ones(dimensions)

    for line in f:
        chunks = line.split(" ")
        idx = _add_word(chunks[0])
        glove[idx] = [float(chunk) for chunk in chunks[1:]]
        if len(_idx_to_word) >= vocab_size:
          break

In [2]:
import sys
import tensorflow as tf
import json
from pprint import pprint
import re
import numpy as np
# from embedding import *
import nltk
import itertools
import random
np.random.seed(0)
orig_stdout = sys.stdout


In [3]:
!pip3 install -U spacy

Requirement already up-to-date: spacy in ./anaconda3/lib/python3.5/site-packages (2.0.18)
thinc 6.12.1 has requirement dill<0.3.0,>=0.2.7, but you'll have dill 0.2.5 which is incompatible.
  Found existing installation: cytoolz 0.8.0
Cannot uninstall 'cytoolz'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.
You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
!python3 -m spacy download en_core_web_lg

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /home/aditi/anaconda3/lib/python3.5/site-packages/en_core_web_lg -->
    /home/aditi/anaconda3/lib/python3.5/site-packages/spacy/data/en_core_web_lg

    You can now load the model via spacy.load('en_core_web_lg')



In [5]:
!pip3 install textacy

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [7]:
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.python.layers.core import Dense

In [8]:
import json
data = json.load(open('/home/aditi/Desktop/6th sem/Computer Graphics/train-v2.0.json'))

In [12]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/aditi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
nltkStopWords = stopwords.words('english')

In [17]:
def extractor(data):
    contexts = []
    qas = []
    for i in range(len(data["data"])):
        for j in range(len(data["data"][i]["paragraphs"])):
            contexts.append(data["data"][i]["paragraphs"][j]["context"])
            qas.append(data["data"][i]["paragraphs"][j]["qas"])
    return (contexts,qas)

In [18]:
CapPassage = False

from nltk.tokenize import word_tokenize
contexts,qas = extractor(data)

def find_sub_list(sl,l):
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            return ind,ind+sll
    return (-1,-1)

def capPassage(passage,answer,cap_length = 30):
    y = np.zeros(cap_length)
    left,right = find_sub_list(answer,passage)
    if(left==-1):
        return passage[0:cap_length]
    left = left - int((cap_length - len(answer))/2)
    right = right + int((cap_length + len(answer))/2)
    if(left < 0):
        left = 0
    if(right > len(passage)):
        right = len(passage)
    return passage[left:right]
    
def findAnsVec(answer,passage):
    ans = np.zeros((len(passage)))
    start,end = find_sub_list(answer,passage)
    if(start==-1):
        start = passage.index(answer[0])
        end = start + len(answer)
    ans[start:end] = 1
    return ans

In [19]:
X_train_comp_all = []
X_train_comp_ans_all = []
X_train_ans_all = []
Y_train_ques_all = []
invalid = 0
X_train_ans_label_all = []
#print(len(contexts))
for i,context in enumerate(contexts):
    passage = word_tokenize(context.lower())
    
    a_lab = np.zeros(len(passage))
    for j,_ in enumerate(qas[i]):
        if (qas[i][j]["answers"]):
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            start,end = find_sub_list(answer,passage)
            if start == -1:
                continue
            a_lab[start:end+1] = 1
            
            
    for j,_ in enumerate(qas[i]):
        try:
            question = word_tokenize(qas[i][j]['question'].lower())
            answer = word_tokenize(qas[i][j]["answers"][0]['text'].lower())
            
            if CapPassage:
                cappedPassage = capPassage(passage,answer)
            else:
                cappedPassage = passage
            
            X_train_comp_ans_all.append(findAnsVec(answer,passage))
            X_train_ans_label_all.append(a_lab)
            X_train_comp_all.append(cappedPassage)
            X_train_ans_all.append(answer)
            Y_train_ques_all.append(question)
        except Exception as e:
            invalid = invalid+1


In [20]:
from collections import Counter
import operator
def findKMostFrequentWords(k):
    ctr = Counter([item for sublist in X_train_comp_all for item in sublist] + [item for sublist in Y_train_ques_all for item in sublist])
    sorted_ctr = sorted(ctr.items(), key=operator.itemgetter(1), reverse=True)
    return [item[0] for item in sorted_ctr[0:k]]

In [21]:
wordsToTake = 80000
words = findKMostFrequentWords(400000)

In [22]:
len(words)

103291

In [23]:
_word_to_idx_reduced = {}
_idx_to_word_reduced = []


def _add_word_reduced(word):
    idx = len(_idx_to_word_reduced)
    _word_to_idx_reduced[word] = idx
    _idx_to_word_reduced.append(word)
    return idx


#PAD_TOKEN = _add_word_reduced(PAD_WORD)
UNKNOWN_TOKEN = _add_word_reduced(UNKNOWN_WORD)
START_TOKEN = _add_word_reduced(START_WORD)
END_TOKEN = _add_word_reduced(END_WORD)




dimensions = glove.shape[1]
reduced_glove = []
reduced_glove.append(np.zeros(dimensions))
reduced_glove.append(-np.ones(dimensions))
reduced_glove.append(np.ones(dimensions))

for word in words:
    l = look_up_word(word)
    if(l != UNKNOWN_TOKEN):
        idx = _add_word_reduced(word)
        reduced_glove.append(glove[l])
    if(len(reduced_glove) == wordsToTake):
        break
        
def look_up_word_reduced(word):
    return _word_to_idx_reduced.get(word, UNKNOWN_TOKEN)


def look_up_token_reduced(token):
    return _idx_to_word_reduced[token]

reduced_glove = np.array(reduced_glove)
reduced_glove.shape

(73218, 100)

In [24]:


!hostname
print(invalid)
for i in np.where(X_train_ans_label_all[110] == 1)[0]:
    print(X_train_comp_all[110][i], end = ' ', sep = ' ')



aditi-Inspiron-13-5368
44984
foxxy cleopatra alongside mike myers in austin powers in goldmember , 73 million . work it out '' uk , norway , and belgium . musical comedy the fighting temptations as mixed reviews from missy elliott , summertime '' 

In [25]:
print(X_train_comp_all[0])
print(X_train_ans_all[0])

['beyoncé', 'giselle', 'knowles-carter', '(', '/biːˈjɒnseɪ/', 'bee-yon-say', ')', '(', 'born', 'september', '4', ',', '1981', ')', 'is', 'an', 'american', 'singer', ',', 'songwriter', ',', 'record', 'producer', 'and', 'actress', '.', 'born', 'and', 'raised', 'in', 'houston', ',', 'texas', ',', 'she', 'performed', 'in', 'various', 'singing', 'and', 'dancing', 'competitions', 'as', 'a', 'child', ',', 'and', 'rose', 'to', 'fame', 'in', 'the', 'late', '1990s', 'as', 'lead', 'singer', 'of', 'r', '&', 'b', 'girl-group', 'destiny', "'s", 'child', '.', 'managed', 'by', 'her', 'father', ',', 'mathew', 'knowles', ',', 'the', 'group', 'became', 'one', 'of', 'the', 'world', "'s", 'best-selling', 'girl', 'groups', 'of', 'all', 'time', '.', 'their', 'hiatus', 'saw', 'the', 'release', 'of', 'beyoncé', "'s", 'debut', 'album', ',', 'dangerously', 'in', 'love', '(', '2003', ')', ',', 'which', 'established', 'her', 'as', 'a', 'solo', 'artist', 'worldwide', ',', 'earned', 'five', 'grammy', 'awards', 'and'

In [26]:
find_sub_list(X_train_ans_all[0] , X_train_comp_all[0])

(50, 54)

In [27]:
print(invalid)
print(X_train_comp_all[101])
print(X_train_ans_all[101])
print(Y_train_ques_all[101])

c = list(zip(X_train_comp_all,X_train_comp_ans_all, X_train_ans_all, X_train_ans_label_all,Y_train_ques_all))
np.random.shuffle(c)
X_train_comp_all_shuffled,X_train_comp_ans_all_shuffled, X_train_ans_shuffled, X_train_ans_label_shuffled,Y_train_ques_all_shuffled = zip(*c)

print(X_train_comp_all_shuffled[101])
print(X_train_comp_ans_all_shuffled[101])
print(X_train_ans_shuffled[101])
print(X_train_ans_label_shuffled[101])
print(Y_train_ques_all_shuffled[101])

44984
['the', 'remaining', 'band', 'members', 'recorded', '``', 'independent', 'women', 'part', 'i', "''", ',', 'which', 'appeared', 'on', 'the', 'soundtrack', 'to', 'the', '2000', 'film', ',', 'charlie', "'s", 'angels', '.', 'it', 'became', 'their', 'best-charting', 'single', ',', 'topping', 'the', 'u.s.', 'billboard', 'hot', '100', 'chart', 'for', 'eleven', 'consecutive', 'weeks', '.', 'in', 'early', '2001', ',', 'while', 'destiny', "'s", 'child', 'was', 'completing', 'their', 'third', 'album', ',', 'beyoncé', 'landed', 'a', 'major', 'role', 'in', 'the', 'mtv', 'made-for-television', 'film', ',', 'carmen', ':', 'a', 'hip', 'hopera', ',', 'starring', 'alongside', 'american', 'actor', 'mekhi', 'phifer', '.', 'set', 'in', 'philadelphia', ',', 'the', 'film', 'is', 'a', 'modern', 'interpretation', 'of', 'the', '19th', 'century', 'opera', 'carmen', 'by', 'french', 'composer', 'georges', 'bizet', '.', 'when', 'the', 'third', 'album', 'survivor', 'was', 'released', 'in', 'may', '2001', ',', 

In [28]:
len(X_train_comp_all)

85335

In [29]:
# Making a tradeoff in examples to train because of memory constraints

examples_to_take_train = 55000

X_train_comp = X_train_comp_all_shuffled[0:examples_to_take_train]
X_train_comp_ans = X_train_comp_ans_all_shuffled[0:examples_to_take_train]
X_train_ans = X_train_ans_shuffled[0:examples_to_take_train]
X_train_ans_label = X_train_ans_label_shuffled[0:examples_to_take_train]
Y_train_ques = Y_train_ques_all_shuffled[0:examples_to_take_train]
answer_indices = [np.where(x==1)[0].tolist() for x in X_train_comp_ans]

In [30]:
max_document_len = len(max(X_train_comp,key=len))
max_answer_len = len(max(X_train_ans,key=len))
max_question_len = len(max(Y_train_ques,key=len)) + 1
print(max_document_len)
print(max_answer_len)
print(max_question_len)

766
46
50


In [31]:
document_tokens = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
document_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
answer_labels = np.zeros((examples_to_take_train, max_document_len), dtype=np.int32)
answer_masks = np.zeros((examples_to_take_train, max_answer_len, max_document_len), dtype=np.int32)
answer_lengths = np.zeros(examples_to_take_train, dtype=np.int32)
question_input_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_output_tokens = np.zeros((examples_to_take_train, max_question_len), dtype=np.int32)
question_lengths = np.zeros(examples_to_take_train, dtype=np.int32)

In [32]:
print(answer_labels.shape)
for i in range(examples_to_take_train):
    answer_labels[i,0:len(X_train_ans_label[i])] = X_train_ans_label[i]
    for j, word in enumerate(X_train_comp[i]):
        document_tokens[i, j] = look_up_word_reduced(word)
    document_lengths[i] = len(X_train_comp[i])

    for j, index in enumerate(answer_indices[i]):
        answer_masks[i, j, index] = 1
    answer_lengths[i] = len(answer_indices[i])
    
    #print(Y_train_ques[i])
    question_input_words = ([START_WORD] + Y_train_ques[i])
    question_output_words = (Y_train_ques[i] + [END_WORD])

    for j, word in enumerate(question_input_words):
            question_input_tokens[i, j] = look_up_word_reduced(word)
    for j, word in enumerate(question_output_words):
        question_output_tokens[i, j] = look_up_word_reduced(word)
    question_lengths[i] = len(question_input_words)

(55000, 766)


In [33]:
print(reduced_glove.shape)

(73218, 100)


In [34]:
def sentences_to_indices_glove(X,max_len):
    
    m = len(X)                                 
    
    X_indices = np.full([m,max_len],look_up_word_reduced(PAD_WORD))
    
    for i in range(m):
        j = 0
        for w in X[i]:
            if(j>=max_len):
                break;
            
            X_indices[i, j] = look_up_word_reduced(w)
            j = j+1
        if(j<max_len):
            X_indices[i,j] = look_up_word_reduced(END_WORD)
    return X_indices

In [35]:
document_tokens = sentences_to_indices_glove(X_train_comp, max_document_len)

In [37]:
document_tokens[0]

array([    8,    33,  3464,  1344,     4,  4643,     8,  1614,   602,
           7,  7467,   130,  4194,    29,   480,    30,    60,  1157,
           9,  2006,  2115,   585,     6,  4874,  2348,     4,  3098,
           7,  6416,    13,   117,    13, 12995,  1645,  1688,     8,
           3,  1670,   217,     4,   359,     3,  1754,     5,  4366,
           4,    33,   215,  2851,     8,    65,   920,  5315,     5,
           3,   155,    72,    84,     4,    85,  3162,    10,  1772,
         336,     6,   101,     4,     3,    94,     5,     0,  3476,
           4,    87,   903,   255,  1328,     4,  1638,    43,   294,
        1390,     4,  2769,   146,    57,  1091,     9, 45276,     6,
         102,  3002,   100,  3795,     4,     3,   130,    43,  7174,
           3,   477,     8,  1942,     7, 10069,    42,   389,  1800,
         480,   129,   294,  5954,     9,    10,  2772,   703,     6,
         765,  7877,    29,   374, 37405,   653,     9,  5038,  2026,
           7, 12679,

In [39]:
answer_labels.shape

(55000, 766)

In [40]:
import math

def createBatch(inputs,batch_size,shuffle=False):
    outputs = []
    start = 0
    while start < len(inputs[0]):
        end = min(len(inputs[0]), start + batch_size)
        output = {'document_tokens':[],
                    'document_lengths':[],
                    'answer_labels':[],
                    'answer_mask': [],
                    'answer_lengths': [],
                    'question_input_tokens':[],
                    'question_output_tokens':[],
                    'question_lengths':[],
                 }
        
        for index,inp in enumerate(inputs):
            maxD = max(inputs[1][start:start+batch_size])
            maxA = max(inputs[4][start:start+batch_size])
            maxQ = max(inputs[7][start:start+batch_size])
            
            if index == 0:
                output['document_tokens'].append(inp[start:end,0:maxD])
            elif index==1:
                output['document_lengths'].append(inp[start:end])
            elif index==2:
                output['answer_labels'].append(inp[start:end,0:maxD])
            elif index==3:
                output['answer_mask'].append(inp[start:end,0:maxA,0:maxD])
            elif index==4:
                output['answer_lengths'].append(inp[start:end])
            elif index==5:
                output['question_input_tokens'].append(inp[start:end, 0:maxQ])
            elif index==6:
                output['question_output_tokens'].append(inp[start:end, 0:maxQ])
            elif index==7:
                output['question_lengths'].append(inp[start:end])
            
        output["document_tokens"] = np.array(output["document_tokens"])
        output["document_lengths"] = np.array(output["document_lengths"])
        output["answer_labels"] = np.array(output["answer_labels"])
        output["answer_mask"] = np.array(output["answer_mask"])
        output["answer_lengths"] = np.array(output["answer_lengths"])
        output["question_input_tokens"] = np.array(output["question_input_tokens"])
        output["question_output_tokens"] = np.array(output["question_output_tokens"])
        output["question_lengths"] = np.array(output["question_lengths"])
        outputs.append(output)
        start = start + batch_size
            
    return outputs

In [41]:
batch_size = 32
batch_input = createBatch([document_tokens,document_lengths,answer_labels,answer_masks,answer_lengths,question_input_tokens,question_output_tokens,question_lengths]
                    ,batch_size)

for b in batch_input:
    for k, v in b.items():
        b[k] = v.squeeze(0)